In [12]:
import os
import subprocess
import shutil
from glob import glob
import nibabel as nib
import nibabel.processing
import numpy as np
import monai
import torch
monai.config.print_config()

MONAI version: 1.3.0
Numpy version: 1.21.6
Pytorch version: 1.13.0+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/.local/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.8.0
Pillow version: 9.5.0
Tensorboard version: 2.8.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.2
einops version: 0.4.1
transformers version: 4.24.0
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: 1.0.0
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:

# Resample Allen Atlas to 128^3 and Origin with MONAI

Atlas downloaded from https://scalablebrainatlas.incf.org/mouse/ABA_v3#downloads

In [14]:
target_affine = np.array([
        [ 0.1200,  0.0000,  0.0000, -7.6800],
        [ 0.0000,  0.1200,  0.0000, -7.6800],
        [ 0.0000,  0.0000,  0.1200, -7.6800],
        [ 0.0000,  0.0000,  0.0000,  1.0000],
])
print(target_affine)
atlas_mri = os.path.join('Atlas', 'P56_Atlas_downsample2.nii.gz')
atlas_label = os.path.join('Atlas', 'P56_Annotation.nii.gz')

#transform_mask = monai.transforms.SpatialResample(mode="nearest")
#transform_mri = monai.transforms.SpatialResample(mode="bilinear")

mri = nib.load(atlas_mri)
mri_out = nib.processing.resample_from_to(mri, ((128,128,128), target_affine))
nib.save(mri_out, atlas_mri.replace('.nii.gz', '_test.nii.gz'))

#loader = monai.transforms.LoadImage(ensure_channel_first=True)
#saver = monai.transforms.SaveImage(output_dir='Atlas', output_postfix='_int128', separate_folder=False) 
#
#transform_mri = monai.transforms.SpatialResample(mode=5)
#tensor = loader(atlas_mri)
#out_mri = transform_mri(tensor, target_affine, spatial_size=(128,128,128))
#saver(out_mri)
#
#ensure_mask = monai.transforms.AsDiscrete(threshold=0.5)
#ensure_label = monai.transforms.AsDiscrete(rounding="torchrounding")
#
#label = loader(atlas_label)
#out_label = transform_mask(label, target_affine, spatial_size=(128,128,128))
#out_label = ensure_label(out_label)
#saver(out_label)
#print('yes')

[[ 0.12  0.    0.   -7.68]
 [ 0.    0.12  0.   -7.68]
 [ 0.    0.    0.12 -7.68]
 [ 0.    0.    0.    1.  ]]


In [8]:
target_affine = torch.tensor([
        [ 0.1200,  0.0000,  0.0000, -7.6800],
        [ 0.0000,  0.1200,  0.0000, -7.6800],
        [ 0.0000,  0.0000,  0.1200, -7.6800],
        [ 0.0000,  0.0000,  0.0000,  1.0000]
], dtype=torch.float64)

atlas_mri = os.path.join('Atlas', 'P56_Atlas_128.nii.gz')
atlas_label = os.path.join('Atlas', 'P56_Annotation_128.nii.gz')

transform = monai.transforms.ScaleIntensity(minv=0.0, maxv=1.0)  

loader = monai.transforms.LoadImage(ensure_channel_first=True)
saver = monai.transforms.SaveImage(output_dir='Atlas', output_postfix='norm', separate_folder=False) 

tensor = loader(atlas_mri)
label = loader(atlas_label)
out = transform(tensor) 
saver(out)
out = atlas_label.replace(".nii", "_norm.nii")
shutil.copyfile(atlas_label, out)


print('yes')

2023-11-08 15:32:34,300 INFO image_writer.py:197 - writing: Atlas/P56_Atlas_128_norm.nii.gz
yes


In [10]:
identity_affine = np.diag((1, 1, 1, 0))

atlas_mri = os.path.join('Atlas', 'P56_Atlas_128_norm.nii.gz')
atlas_label = os.path.join('Atlas', 'P56_Annotation_128_norm.nii.gz')

loader = monai.transforms.LoadImage(ensure_channel_first=True)    
saver = monai.transforms.SaveImage(output_dir='Atlas', output_postfix='id', separate_folder=False)

tensor = loader(atlas_label)
tensor.affine = identity_affine
saver(tensor)    

tensor = loader(atlas_mri)
tensor.affine = identity_affine
saver(tensor)        
print('yes')

2023-11-08 15:55:39,650 INFO image_writer.py:197 - writing: Atlas/P56_Annotation_128_norm_id.nii.gz
2023-11-08 15:55:39,772 INFO image_writer.py:197 - writing: Atlas/P56_Atlas_128_norm_id.nii.gz
yes


# Create Mask from Labels for Allen Atlas

In [5]:
atlas_label = os.path.join('Atlas', 'P56_Annotation_128_norm_id.nii.gz')

transform = monai.transforms.AsDiscrete(threshold=0.99)

loader = monai.transforms.LoadImage(ensure_channel_first=True)    
saver = monai.transforms.SaveImage(output_dir='Atlas', output_postfix='mask', separate_folder=False)

tensor = loader(atlas_label)
out = transform(tensor)
saver(out)
print('yes')

2023-11-26 10:53:44,118 INFO image_writer.py:197 - writing: Atlas/P56_Annotation_128_norm_id_mask.nii.gz
yes
